In [ ]:
# Google Colab上で実行

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
cd /content/drive/MyDrive/interface

/content/drive/MyDrive/interface


In [ ]:
import pandas as pd
data = pd.read_csv('Data/ModeSelectVector03.csv', names=('x1', 'x2', 'x3', 'x4', 'y'))
data_x = data[['x1', 'x2', 'x3', 'x4']]
data_y = data['y'].to_numpy()

In [ ]:
import numpy as np
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

import torch
import torch.nn.functional as F
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset

# 0. 入出力データの次元
input_dimension = 4
output_dimension = 3

# 1. データ準備、前処理
x_train, x_valid, y_train, y_valid = train_test_split(data_x, data_y, shuffle=True)

# 特徴量の標準化
scaler = StandardScaler()
scaler.fit(x_train)

x_train = scaler.transform(x_train)
x_valid = scaler.transform(x_valid)

# Tensor型に変換
x_train = torch.from_numpy(x_train).float()
y_train = torch.from_numpy(y_train).long()
x_valid = torch.from_numpy(x_valid).float()
y_valid = torch.from_numpy(y_valid).long()

print('x_train : ', x_train.shape)
print('y_train : ', y_train.shape)
print('x_valid : ', x_valid.shape)
print('y_valid : ', y_valid.shape)

# 2. Datasetの作成
train_dataset = TensorDataset(x_train, y_train)
valid_dataset = TensorDataset(x_valid, y_valid)

# 3. DataLoaderの作成
batch_size = 32
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
valid_dataloader = DataLoader(valid_dataset, batch_size=batch_size, shuffle=False)

# 4. ネットワークの定義
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(input_dimension, 30)
        self.fc2 = nn.Linear(30, 20)
        self.fc3 = nn.Linear(20, output_dimension)

    def forward(self, x):
        x = self.fc1(x)
        x = F.relu(x)
        x = self.fc2(x)
        x = F.relu(x)
        x = self.fc3(x)
        x = F.softmax(x, dim=1)
        return x

net = Net()
print(net)

# 5. 損失関数の定義
criterion = nn.CrossEntropyLoss()

# 6. 最適化手法の定義
optimizer = optim.SGD(net.parameters(), lr=0.01)

# 7. 学習・評価
num_epochs = 300

# 学習時と検証時で分けるためディクショナリを用意
dataloaders_dict = {
    'train': train_dataloader,
    'val': valid_dataloader
}

for epoch in range(num_epochs):
    for phase in ['train', 'val']:
        if phase == 'train':
            net.train()
        else:
            net.eval()

        # 損失和
        epoch_loss = 0.0
        # 正解数
        epoch_corrects = 0

        for inputs, labels in dataloaders_dict[phase]:
            # optimizerの初期化
            optimizer.zero_grad()

            # 学習時のみ勾配を計算させる設定にする
            with torch.set_grad_enabled(phase == 'train'):
                outputs = net(inputs)
                loss = criterion(outputs, labels)
                _, preds = torch.max(outputs, 1)

                # 訓練時はバックプロパゲーション
                if phase == 'train':
                    loss.backward()
                    optimizer.step()

                epoch_loss += loss.item() * inputs.size(0)

                # 正解数の合計を更新
                epoch_corrects += torch.sum(preds == labels.data)

        # epochごとのlossと正解率を表示
        if (epoch + 1) % 20 != 0:
          continue
        epoch_loss = epoch_loss / len(dataloaders_dict[phase].dataset)
        epoch_acc = epoch_corrects.double() / len(dataloaders_dict[phase].dataset)

        print('{} Loss: {:.4f} Acc: {:.4f}'.format(phase, epoch_loss, epoch_acc))

x_train :  torch.Size([560, 4])
y_train :  torch.Size([560])
x_valid :  torch.Size([187, 4])
y_valid :  torch.Size([187])
Net(
  (fc1): Linear(in_features=4, out_features=30, bias=True)
  (fc2): Linear(in_features=30, out_features=20, bias=True)
  (fc3): Linear(in_features=20, out_features=3, bias=True)
)
train Loss: 0.9732 Acc: 0.8268
val Loss: 0.9663 Acc: 0.8717
train Loss: 0.7889 Acc: 0.8536
val Loss: 0.7766 Acc: 0.8770
train Loss: 0.7102 Acc: 0.8536
val Loss: 0.6956 Acc: 0.8770
train Loss: 0.6807 Acc: 0.8536
val Loss: 0.6696 Acc: 0.8824
train Loss: 0.6549 Acc: 0.9161
val Loss: 0.6499 Acc: 0.9144
train Loss: 0.6362 Acc: 0.9161
val Loss: 0.6350 Acc: 0.9144
train Loss: 0.6245 Acc: 0.9161
val Loss: 0.6247 Acc: 0.9144
train Loss: 0.6147 Acc: 0.9304
val Loss: 0.6155 Acc: 0.9144
train Loss: 0.6043 Acc: 0.9786
val Loss: 0.6053 Acc: 0.9679
train Loss: 0.5927 Acc: 0.9946
val Loss: 0.5937 Acc: 0.9947
train Loss: 0.5817 Acc: 1.0000
val Loss: 0.5827 Acc: 1.0000
train Loss: 0.5732 Acc: 1.0000
va

In [ ]:
print(net)

Net(
  (fc1): Linear(in_features=4, out_features=30, bias=True)
  (fc2): Linear(in_features=30, out_features=20, bias=True)
  (fc3): Linear(in_features=20, out_features=3, bias=True)
)


In [ ]:
pip install onnx

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.6/14.6 MB 68.8 MB/s eta 0:00:00


In [ ]:
x = torch.randn(1, input_dimension)

# モデルのエクスポート
torch.onnx.export(net,         # モデル
    x,                         # モデルの入力
    "ONNX/ModeSelect03.onnx",            # ONNXファイル名
    export_params=True,        # 重みをONNXファイルに保存
    opset_version=9,           # ONNXのバージョン
    do_constant_folding=True,  # 最適化のために定数の折りたたみの実行
    input_names = ['X'],       # モデルの入力名
    output_names = ['Y']       # モデルの出力名
    )

============= Diagnostic Run torch.onnx.export version 2.0.1+cu118 =============
verbose: False, log level: Level.ERROR
======================= 0 NONE 0 NOTE 0 WARNING 0 ERROR ========================

